compare calibration result and simulation result in the instant loop detector

In [ ]:
import os
print("Current directory:", os.getcwd())
HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

#number = len(data)
import pandas as pd
date = '2020-01-01'
detector = 'e2w_in'
path  = "data/sim_data/"
postfix = f"{detector}_{date}"
instantInductionLoop_filename_xml = f"{path}instantInductionLoop_{postfix}.xml"


Current directory: /home/kaveh/projects/Hornsgatan
Current directory: /home/kaveh/projects/Hornsgatan


In [109]:
import xml.etree.ElementTree as ET

# Parse the XML content of output_detectors.xml
tree_output = ET.parse(instantInductionLoop_filename_xml)
root_output = tree_output.getroot()

# Extract data from the XML
instant_out_data = []
for instant_out in root_output.findall('instantOut'):
    instant_out_data.append({
        'id': instant_out.get('id'),
        'time': float(instant_out.get('time')),
        'state': instant_out.get('state'),
        'vehID': instant_out.get('vehID'),
        'speed': float(instant_out.get('speed')),  # Convert speed to km/h
        'length': instant_out.get('length'),
        'type': instant_out.get('type'),
        'gap': instant_out.get('gap')
    })

# Convert to a DataFrame
df_instant_out = pd.DataFrame(instant_out_data)
df_instant_out = df_instant_out[df_instant_out["state"]=='enter']
# Save to CSV
#output_csv_file_instant_out = 'output_instant_out.csv'
#df_instant_out.to_csv(output_csv_file_instant_out, index=False)
#print(f"Data from 'output_detectors.xml' successfully converted to CSV and saved as '{output_csv_file_instant_out}'.")


In [110]:
df_instant_out

,id,time,state,vehID,speed,length,type,gap
0,e2w_out,1.577837e+09,enter,0_e2w_out,8.33,5.00,DEFAULT_VEHTYPE,None
2,e2w_out,1.577837e+09,enter,1_e2w_out,10.83,5.00,DEFAULT_VEHTYPE,28.85
5,e2w_out,1.577837e+09,enter,2_e2w_out,7.50,5.00,DEFAULT_VEHTYPE,36.23
8,e2w_out,1.577837e+09,enter,3_e2w_out,8.33,5.00,DEFAULT_VEHTYPE,36.18
10,e2w_out,1.577837e+09,enter,4_e2w_out,10.00,5.00,DEFAULT_VEHTYPE,6.67
...,...,...,...,...,...,...,...,...
2420,e2w_out,1.577923e+09,enter,916_e2w_out,8.33,5.00,DEFAULT_VEHTYPE,284.67
2422,e2w_out,1.577923e+09,enter,917_e2w_out,8.33,5.00,DEFAULT_VEHTYPE,3.40
2424,e2w_out,1.577923e+09,enter,918_e2w_out,10.83,5.00,DEFAULT_VEHTYPE,59.85
2427,e2w_out,1.577923e+09,enter,919_e2w_out,8.33,5.00,DEFAULT_VEHTYPE,137.08


In [111]:
data = pd.read_csv(f'data/calibration_data/calibrated_data_{postfix}.csv')
data.rename(columns={"veh_id": "vehID"}, inplace=True)
data.head()

,vehID,time_detector_sim,speed_detector_sim,speed_factor,time_detector_real,depart,departSpeed,speed_detector_real,delta_time,delta_speed
0,0_e2w_out,1.577837e+09,8.330001,1.0,1577836850,1.577837e+09,8.330,7.777778,0.36,0.552224
1,1_e2w_out,1.577837e+09,10.829000,1.3,1577836881,1.577837e+09,10.829,11.111111,-1.18,-0.282111
2,2_e2w_out,1.577837e+09,7.497000,0.9,1577836917,1.577837e+09,7.497,3.611111,-0.49,3.885889
3,3_e2w_out,1.577837e+09,8.330001,1.0,1577836953,1.577837e+09,8.330,9.166667,0.36,-0.836665
4,4_e2w_out,1.577837e+09,9.996000,1.2,1577836960,1.577837e+09,9.996,9.722222,0.63,0.273778


In [112]:
print(len(data), len(df_instant_out))

921 921


In [113]:
compare_df = pd.merge(df_instant_out[["vehID","time","speed"]],data[["vehID","time_detector_sim","speed_detector_sim"]],on="vehID")
compare_df

,vehID,time,speed,time_detector_sim,speed_detector_sim
0,0_e2w_out,1.577837e+09,8.33,1.577837e+09,8.330001
1,1_e2w_out,1.577837e+09,10.83,1.577837e+09,10.829000
2,2_e2w_out,1.577837e+09,7.50,1.577837e+09,7.497000
3,3_e2w_out,1.577837e+09,8.33,1.577837e+09,8.330001
4,4_e2w_out,1.577837e+09,10.00,1.577837e+09,9.996000
...,...,...,...,...,...
916,916_e2w_out,1.577923e+09,8.33,1.577923e+09,8.330001
917,917_e2w_out,1.577923e+09,8.33,1.577923e+09,8.330001
918,918_e2w_out,1.577923e+09,10.83,1.577923e+09,10.829000
919,919_e2w_out,1.577923e+09,8.33,1.577923e+09,8.330001


In [114]:
time_test = (compare_df["time"]-compare_df["time_detector_sim"])
time_test = abs(time_test)

In [115]:
time_test = time_test[time_test>1]

In [116]:
time_test

20     2.63
37     1.98
42     1.92
48     1.66
53     1.04
       ... 
889    2.72
897    2.91
900    2.68
903    1.15
912    1.15
Length: 85, dtype: float64

In [117]:
len(time_test)

85

In [118]:
speed_test = (compare_df["speed"]-compare_df["speed_detector_sim"])
abs(speed_test).sum()

np.float64(55.95203750387298)

In [119]:
speed_test = abs(speed_test)

In [120]:
speed_test = speed_test[speed_test>0.01]
len(speed_test)

48

In [121]:
speed_test

37     1.471489
42     1.692504
44     0.375662
71     0.620922
72     0.833000
93     0.663399
96     1.143986
134    0.961971
143    1.988710
188    1.569483
228    1.534765
279    0.413500
285    1.548172
345    1.669001
352    1.413727
365    1.011984
368    0.253400
371    1.475550
380    1.801775
381    2.148372
428    1.104725
445    1.874856
452    0.633084
463    2.004427
472    0.995598
490    0.335848
533    1.253000
545    0.056001
568    1.736299
591    0.836001
632    0.170000
647    1.960353
670    1.501235
686    0.835996
714    0.476587
755    1.192731
790    1.803585
803    0.850664
815    0.420000
821    0.170000
856    0.832001
861    0.832013
887    0.828996
889    1.267520
890    0.806831
900    1.353577
903    1.523732
912    1.962823
dtype: float64

compare time error and speed error in loop detector between simulation result and calibration result

In [122]:
compare_df = pd.merge(df_instant_out[["vehID","time","speed"]],
            data[["vehID","time_detector_sim","speed_detector_sim"
                  ,"time_detector_real","speed_detector_real"]],on="vehID")
compare_df.head()


,vehID,time,speed,time_detector_sim,speed_detector_sim,time_detector_real,speed_detector_real
0,0_e2w_out,1.577837e+09,8.33,1.577837e+09,8.330001,1577836850,7.777778
1,1_e2w_out,1.577837e+09,10.83,1.577837e+09,10.829000,1577836881,11.111111
2,2_e2w_out,1.577837e+09,7.50,1.577837e+09,7.497000,1577836917,3.611111
3,3_e2w_out,1.577837e+09,8.33,1.577837e+09,8.330001,1577836953,9.166667
4,4_e2w_out,1.577837e+09,10.00,1.577837e+09,9.996000,1577836960,9.722222


In [123]:
time_calib_error = compare_df["time_calib_error"]=(compare_df["time_detector_sim"]-compare_df["time_detector_real"])
abs(time_calib_error).mean()

np.float64(0.9222584050330743)

In [124]:
time_sim_error = compare_df["time_sim_error"] = compare_df["time"]-compare_df["time_detector_real"]
abs(time_sim_error).mean()

np.float64(0.766894670314561)

In [125]:
speed_calib_error= compare_df["speed_calib_error"] =compare_df["speed_detector_sim"]-compare_df["speed_detector_real"]
abs(speed_calib_error).mean()

np.float64(0.970606250774964)

In [126]:
speed_sim_error=compare_df["speed_sim_error"] =compare_df["speed"]-compare_df["speed_detector_real"]
abs(speed_sim_error).mean()

np.float64(1.0021401857883943)

In [127]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-4,  # Bin start
        end=11,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['time_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['time_calib_error']).mean(),2)

fig.add_trace(go.Histogram(x=compare_df['time_sim_error'], name=f"time_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['time_calib_error'], name=f"time_calib_error  --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='time error (s)',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_time_error_{postfix}.html")

In [128]:
print(compare_df['time_sim_error'].min(), compare_df['time_sim_error'].max())
print(compare_df['time_calib_error'].min(), compare_df['time_calib_error'].max())

-3.359999895095825 8.940000057220459
-1.9100000858306885 10.579999923706055


In [129]:
print(compare_df['speed_sim_error'].min(), compare_df['speed_sim_error'].max())
print(compare_df['speed_calib_error'].min(), compare_df['speed_calib_error'].max())

-6.666666666666666 6.111111111111111
-4.757534516970681 6.108111111111111


In [130]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-7,  # Bin start
        end=7,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['speed_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['speed_calib_error']).mean(),2)
fig.add_trace(go.Histogram(x=compare_df['speed_sim_error'], name=f"speed_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['speed_calib_error'], name=f"speed_calib_error --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='Value',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_speed_error_{postfix}.html")

environ({'USER': 'kaveh', 'SSH_CLIENT': '50.98.120.254 52394 22', 'LC_TIME': 'sv_SE.UTF-8', 'XDG_SESSION_TYPE': 'tty', 'SHLVL': '1', 'MOTD_SHOWN': 'pam', 'HOME': '/home/kaveh', 'OLDPWD': '/home/kaveh/.vscode-server', 'LC_MONETARY': 'sv_SE.UTF-8', 'SSL_CERT_FILE': '/usr/lib/ssl/certs/ca-certificates.crt', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1007/bus', 'LOGNAME': 'kaveh', '_': '/home/kaveh/calib_venv/bin/python', 'XDG_SESSION_CLASS': 'user', 'XDG_SESSION_ID': '8314', 'VSCODE_CLI_REQUIRE_TOKEN': 'ae03dc89-2af2-452f-9f46-712bcd09a747', 'PATH': '/home/kaveh/calib_venv/bin:/home/kaveh/.vscode-server/cli/servers/Stable-848b80aeb52026648a8ff9f7c45a9b0a80641e2e/server/bin/remote-cli:/home/kaveh/.local/bin:/home/kaveh/anaconda3/bin:/home/kaveh/anaconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin', 'VSCODE_AGENT_FOLDER': '/home/kaveh/.vscode-server', 'LC_ADDRESS': 'sv_SE.UTF-8', 'XDG_RUNTIME_DIR': '/run/user/1007', 'SSL

In [1]:
import os
os.environ["HORNSGATAN_HOME"]


'/home/kaveh/Hornsgatan/'